<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/text_classification/spam_vs_ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# upload kaggle API key from your local machine
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shazzadraihan","key":"da63bbe0f8dcb3bd7fb35034046ca758"}'}

In [ ]:
# make a kaggle dir, copy the API key to it
# and make sure the file in only readable by yourself (chmod 600)
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# use API command to download the dataset
!kaggle datasets download -d uciml/sms-spam-collection-dataset

  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 69.7MB/s]


In [ ]:
# uncompress the dataset
!unzip -qq sms-spam-collection-dataset.zip

In [ ]:
!pip install torchtext==0.9.1
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 2.1 MB/s 
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [ ]:
import torch
import torchtext
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print("CUDA is not available")
else:
  print("CUDA is available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

CUDA is available


### Load and Visualize the data

In [ ]:
data = pd.read_csv("/content/spam.csv", encoding="latin-1")

data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### Data pre-processing

In [ ]:
data = data.drop(columns = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(index = str, columns = {"v1": "labels", "v2": "text"})

data.head()

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
train, test = train_test_split(data, test_size = 0.2, random_state = 42)

train.reset_index(drop=True), test.reset_index(drop=True)

(     labels                                               text
 0       ham  No I'm in the same boat. Still here at my moms...
 1      spam  (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
 2       ham     They r giving a second chance to rahul dengra.
 3       ham     O i played smash bros  &lt;#&gt;  religiously.
 4      spam  PRIVATE! Your 2003 Account Statement for 07973...
 ...     ...                                                ...
 4452    ham  I came hostel. I m going to sleep. Plz call me...
 4453    ham                             Sorry, I'll call later
 4454    ham      Prabha..i'm soryda..realy..frm heart i'm sory
 4455    ham                         Nt joking seriously i told
 4456    ham                In work now. Going have in few min.
 
 [4457 rows x 2 columns],
      labels                                               text
 0       ham  Funny fact Nobody teaches volcanoes 2 erupt, t...
 1       ham  I sent my scores to sophas and i had to do sec...
 2      spam

In [ ]:
train.shape, test.shape

((4457, 2), (1115, 2))

In [ ]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

In [ ]:
!ls

kaggle.json  sms-spam-collection-dataset.zip  test.csv
sample_data  spam.csv			      train.csv


In [ ]:
import nltk
nltk.download("punkt")

from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
TEXT = Field(tokenize = word_tokenize)

In [ ]:
LABEL = Field(dtype = torch.float)

In [ ]:
datafields = [("labels", LABEL), ("text", TEXT)]

In [ ]:
trn, tst = TabularDataset.splits(path = './',
                                        train = "train.csv",
                                        test = "test.csv",
                                        format = "csv",
                                        skip_header = True,
                                        fields = datafields)

In [ ]:
trn[:5]

In [ ]:
print(f'Number of training examples: {len(trn)}')
print(f'Number of testing examples: {len(tst)}')

Number of training examples: 4457
Number of testing examples: 1115


In [ ]:
trn[5].__dict__.keys()

dict_keys(['labels', 'text'])

In [ ]:
trn[5].text

['G', 'says', 'you', 'never', 'answer', 'your', 'texts', ',', 'confirm/deny']

In [ ]:
trn[5].labels

['ham']

In [ ]:
print(vars(trn.examples[5]))

{'labels': ['ham'], 'text': ['G', 'says', 'you', 'never', 'answer', 'your', 'texts', ',', 'confirm/deny']}


In [ ]:
TEXT.build_vocab(trn, max_size = 10500)

In [ ]:
LABEL.build_vocab(trn)

In [ ]:
print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}')

Unique tokens in TEXT vocabulary: 10213
Unique tokens in LABEL vocabulary: 4


In [ ]:
print(TEXT.vocab.freqs.most_common(50))

[('.', 3847), ('to', 1750), ('I', 1574), (',', 1468), ('you', 1462), ('?', 1256), ('!', 1134), ('a', 1068), ('the', 946), ('...', 923), ('&', 772), ('i', 760), ('and', 673), ('in', 663), ('is', 647), (';', 641), ('u', 636), ('me', 600), (':', 570), ('..', 544), ('for', 527), ('my', 494), ('of', 471), ('it', 470), ('your', 461), ('have', 395), ('on', 394), (')', 393), ('2', 390), ('that', 385), ("'s", 384), ("'m", 320), ('now', 318), ('are', 316), ('do', 312), ('call', 307), ('at', 301), ('U', 300), ('or', 298), ('not', 295), ("n't", 281), ('be', 275), ('*', 270), ('lt', 267), ('gt', 267), ('with', 267), ('get', 265), ('will', 264), ('so', 257), ('#', 245)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '.', 'to', 'I', ',', 'you', '?', '!', 'a']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7ff39b873790>>, {'<unk>': 0, '<pad>': 1, 'ham': 2, 'spam': 3})


### Split the dataset

In [ ]:
batch_size = 64

train_iterator, test_iterator = BucketIterator.splits(
    (trn, tst),
    batch_size = batch_size,
    device = device,
    sort_key = lambda x: len(x.text),
    sort_within_batch = False)

### Define a Model

In [ ]:
 import torch.nn as nn
 import torch.optim as optim

 class RNN(nn.Module):
   def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
     super().__init__()
     self.embedding = nn.Embedding(input_dim, embedding_dim)
     self.rnn = nn.RNN(embedding_dim, hidden_dim)
     self.fc = nn.Linear(hidden_dim, output_dim)

   def forward(self, text):
     embedded = self.embedding(text)
     output, hidden = self.rnn(embedded)
     hidden_1D = hidden.squeeze(0)
     assert torch.equal(output[-1, :, :], hidden_1D)
     return self.fc(hidden_1D)


class LSTM(nn.Module):
  def __init__(self, num_embeddings, embedding_dim, hidden_dim, output_dim, num_layers):
    super(LSTM, self).__init__()

    self.embedding = nn.Embedding(num_embeddings, embedding_dim)
    self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, seq):
    output, (hidden,_) = self.encoder(self.embedding(seq))
    preds = self.fc(hidden.squeeze(0))
    return preds

In [ ]:
num_embeddings  = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1
num_layers = 1

RNN_model = RNN(num_embeddings , embedding_dim, hidden_dim, output_dim)
RNN_model.to(device)

LSTM_model = LSTM(num_embeddings, embedding_dim, hidden_dim, output_dim, num_layers)
LSTM_model.to(device)

LSTM(
  (embedding): Embedding(10213, 100)
  (encoder): LSTM(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

### Train the model

In [ ]:
def train(model, iterator, optimizer, criterion):  

  train_loss = 0.0

  model.train()
  for batch_idx, batch in enumerate(train_iterator):
    optimizer.zero_grad()
    predictions = model(batch.text)
    loss = criterion(predictions.T, batch.labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  train_loss /= len(train_iterator)

  return train_loss

In [ ]:
optimizer = optim.Adam(RNN_model.parameters(), lr = 1e-6)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
  train_loss = train(RNN_model, train_iterator, optimizer, criterion)

  print('Epoch: {} | Training Loss: {:.2f}'.format(epoch, train_loss)) 

  train_losses.append(train_loss)

Epoch: 0 | Training Loss: 0.59
Epoch: 1 | Training Loss: 0.45
Epoch: 2 | Training Loss: 0.32
Epoch: 3 | Training Loss: 0.19
Epoch: 4 | Training Loss: 0.06


In [ ]:
optimizer = optim.Adam(LSTM_model.parameters(), lr = 1e-6)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
  train_loss = train(LSTM_model, train_iterator, optimizer, criterion)

  print('Epoch: {} | Training Loss: {:.2f}'.format(epoch, train_loss)) 

  train_losses.append(train_loss)

Epoch: 0 | Training Loss: 0.75
Epoch: 1 | Training Loss: 0.67
Epoch: 2 | Training Loss: 0.60
Epoch: 3 | Training Loss: 0.52
Epoch: 4 | Training Loss: 0.45


### Test the model